# CSV + API

In this reboot, we are going to use:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal of this livecode is to load the data from a CSV + loop over rows to enrich each row with information such as:

- List of subjects (Science, Humor, Travel, etc.)
- The cover URL of the book
- Other information you'd find useful in the JSON API

First, download the CSV in the local folder:

In [1]:
!curl -L https://gist.githubusercontent.com/ssaunier/351b17f5a7a009808b60aeacd1f4a036/raw/books.csv > books.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1509k  100 1509k    0     0   620k      0  0:00:02  0:00:02 --:--:--  620k


In [2]:
!ls -lh

total 4184
-rw-r--r--  1 luciengeorge  staff   580B Oct 13 10:59 README.md
-rw-r--r--  1 luciengeorge  staff   1.5M Oct 13 11:03 books.csv
-rw-r--r--  1 luciengeorge  staff   2.0K Oct 13 10:59 reboot.ipynb


Then import the usual suspects!

In [3]:
import requests
import pandas as pd
import numpy as np

## Load the data using `pd.read_csv()`

In [11]:
# Your turn!
books_df = pd.read_csv('books.csv')

In [12]:
books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


In [9]:
books_df.dtypes

title          object
authors        object
isbn13          int64
# num_pages     int64
dtype: object

## Fetch `cover_url` from Open Library Books API

In [69]:
def fetch_book_cover_url(isbn13):
    url = "https://openlibrary.org/api/books"
    params = {
        'bibkeys': f'ISBN:{isbn13}',
        'format': 'json',
        'jscmd': 'data'
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data.get(f"ISBN:{isbn13}"):
        return data[f"ISBN:{isbn13}"].get('cover', {}).get('large', 'N/A')
    return 'N/A'

In [70]:
fetch_book_cover_url('9780439385969')

'N/A'

In [64]:
books_df['cover_url'] = None

In [75]:
%%time
# IDEMPOTENT SCRIPT
for index, row in books_df.head(50).iterrows():
    if row['cover_url'] is None:
        print(row['title'])
        cover_url = fetch_book_cover_url(row['isbn13'])
        books_df.loc[index, 'cover_url'] = cover_url

CPU times: user 6.66 ms, sys: 508 µs, total: 7.17 ms
Wall time: 8.95 ms


In [68]:
books_df.head(15)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count,cover_url
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249,https://covers.openlibrary.org/b/id/9326654-L.jpg
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613,https://covers.openlibrary.org/b/id/9326212-L.jpg
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390,https://covers.openlibrary.org/b/id/7572543-L.jpg
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272,https://covers.openlibrary.org/b/id/10301720-L...
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964,https://covers.openlibrary.org/b/id/8778528-L.jpg
5,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,38872,154,https://covers.openlibrary.org/b/id/278981-L.jpg
6,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,0976540606,9780976540601,en-US,152,18,1,https://covers.openlibrary.org/b/id/742235-L.jpg
7,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,27410,820,https://covers.openlibrary.org/b/id/279436-L.jpg
8,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3602,258,https://covers.openlibrary.org/b/id/321859-L.jpg
9,13,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,0345453743,9780345453747,eng,815,240189,3954,None


## Fetch multiple books at once

In [81]:
def fetch_books(isbns):
    url = "https://openlibrary.org/api/books"
    params = {
        'bibkeys': ",".join([f"ISBN:{isbn}" for isbn in isbns]),
        'format': 'json',
        'jscmd': 'data'
    }
    response = requests.get(url, params=params)
    return response.json()

In [82]:
isbns = [9780439785969, 9780439358071, 9780439554893]
fetch_books(isbns)

{'ISBN:9780439785969': {'publishers': [{'name': 'Scholastic'}],
  'links': [{'url': 'https://www.jkrowling.com/book/harry-potter-half-blood-prince/',
    'title': 'jkrowling.com/book/harry-potter-half-blood-prince/'},
   {'url': 'https://en.wikipedia.org/wiki/Harry_Potter_and_the_Half-Blood_Prince',
    'title': 'Wikipedia'},
   {'url': 'https://www.theguardian.com/books/2005/jul/23/booksforchildrenandteenagers.harrypotter',
    'title': 'Into the gloom - review | the Guardian'}],
  'title': 'Harry Potter and the Half-Blood Prince',
  'url': 'https://openlibrary.org/books/OL24280830M/Harry_Potter_and_the_Half-Blood_Prince',
  'notes': 'USA',
  'identifiers': {'openlibrary': ['OL24280830M'],
   'isbn_13': ['9780439785969'],
   'amazon': ['0439785960'],
   'isbn_10': ['0439785960'],
   'oclc': ['70666878', '819153929'],
   'goodreads': ['53178655']},
  'cover': {'small': 'https://covers.openlibrary.org/b/id/9326654-S.jpg',
   'large': 'https://covers.openlibrary.org/b/id/9326654-L.jpg',


In [83]:
books_df['cover_url'] = None

In [85]:
books_df.set_index('isbn13', inplace=True)

In [86]:
books_df

,bookID,title,authors,average_rating,isbn,language_code,# num_pages,ratings_count,text_reviews_count,cover_url
isbn13,,,,,,,,,,
9780439785969,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,eng,652,1944099,26249,None
9780439358071,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,eng,870,1996446,27613,None
9780439554930,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,eng,320,5629932,70390,None
9780439554893,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,eng,352,6267,272,None
9780439655484,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,eng,435,2149872,33964,None
9780439682589,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,0439682584,eng,2690,38872,154,None
9780976540601,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,0976540606,en-US,152,18,1,None
9780439827607,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,eng,3342,27410,820,None
9780517226957,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,eng,815,3602,258,None


## Splitting the df into groups

In [92]:
np.array_split(books_df.head(12), 3)[0]

,bookID,title,authors,average_rating,isbn,language_code,# num_pages,ratings_count,text_reviews_count,cover_url
isbn13,,,,,,,,,,
9780439785969,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,eng,652,1944099,26249,None
9780439358071,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,eng,870,1996446,27613,None
9780439554930,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,eng,320,5629932,70390,None
9780439554893,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,eng,352,6267,272,None


In [107]:
for group in np.array_split(books_df.head(100), 10):
    book_isbns = list(group.index)
    print(f"fetching covers for {book_isbns}")
    books = fetch_books(book_isbns)
    for isbn_code, book in books.items():
        isbn = isbn_code.strip('ISBN:')
        books_df.loc[int(isbn), 'cover_url'] = book.get('cover', {}).get('large', 'N/A')

fetching covers for [9780439785969, 9780439358071, 9780439554930, 9780439554893, 9780439655484, 9780439682589, 9780976540601, 9780439827607, 9780517226957, 9780345453747]
fetching covers for [9781400052929, 9780739322208, 9780517149256, 9780767908184, 9780767915069, 9780767910439, 9780767903868, 9780767903820, 9780060920081, 9780380713806]
fetching covers for [9780380727506, 9780380715435, 9780345538376, 9780618517657, 9780618346240, 9780618346257, 9780618260584, 9780618391004, 9780618510825, 9780618153978]
fetching covers for [9781933372013, 9780976694007, 9780689840920, 9781557344496, 9780385326506, 9781575606248, 9781595580276, 9781595962805, 9780670059676, 9780141312620]
fetching covers for [9780595321803, 9781590301944, 9780449146972, 9780061159176, 9780060762735, 9780060749910, 9780273704744, 9781932386103, 9780965136716, 9780374517199]
fetching covers for [9780374280390, 9780374519742, 9780374522599, 9780374518738, 9780374522872, 9780374519322, 9780374516000, 9780374520656, 9780

In [108]:
books_df.head(100)

,bookID,title,authors,average_rating,isbn,language_code,# num_pages,ratings_count,text_reviews_count,cover_url
isbn13,,,,,,,,,,
9780439785969,1.0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,eng,652.0,1944099.0,26249.0,https://covers.openlibrary.org/b/id/9326654-L.jpg
9780439358071,2.0,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,eng,870.0,1996446.0,27613.0,https://covers.openlibrary.org/b/id/9326212-L.jpg
9780439554930,3.0,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,eng,320.0,5629932.0,70390.0,https://covers.openlibrary.org/b/id/7572543-L.jpg
9780439554893,4.0,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,eng,352.0,6267.0,272.0,https://covers.openlibrary.org/b/id/10301720-L...
9780439655484,5.0,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,eng,435.0,2149872.0,33964.0,https://covers.openlibrary.org/b/id/8778528-L.jpg
9780439682589,8.0,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,0439682584,eng,2690.0,38872.0,154.0,https://covers.openlibrary.org/b/id/278981-L.jpg
9780976540601,9.0,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,0976540606,en-US,152.0,18.0,1.0,https://covers.openlibrary.org/b/id/742235-L.jpg
9780439827607,10.0,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,eng,3342.0,27410.0,820.0,https://covers.openlibrary.org/b/id/279436-L.jpg
9780517226957,12.0,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,eng,815.0,3602.0,258.0,https://covers.openlibrary.org/b/id/321859-L.jpg
